In [1]:
# coding: utf-8

import numpy as np
import torch
import os
import time

# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline  

from networks import *
from network import CoordRegressionNetwork
from torch.utils.data import DataLoader
from dataset_factory import DatasetFactory, ROOT_DIR
import multiprocessing
from tqdm import tqdm

# gpu setting
os.environ["CUDA_VISIBLE_DEVICES"]="1"
torch.backends.cudnn.enabled = True
gpus = [0,1]
device = torch.device("cuda" if True else "cpu")
num_threads = multiprocessing.cpu_count()
print("GPU NUM: %d, Thread NUM: %d"%(torch.cuda.device_count(),num_threads))

%load_ext autoreload
%autoreload 2

GPU NUM: 1, Thread NUM: 20


In [ ]:
# load pretrain model
# modelpath = "./models/resnet18_224_new_best.t7"
modelpath = "./models/senet18_224_new_sgd_best.t7"

if 'resnet18' in modelpath:
    modeltype = "resnet"
    net = CoordRegressionNetwork(n_locations=16, backbone="resnet18").to(device)
    inputsize = 224
elif 'senet18' in modelpath:
    modeltype = "resnet"
    net = CoordRegressionNetwork(n_locations=16, backbone="senet18").to(device)
    inputsize = 224

train_dataset = DatasetFactory.get_train_dataset(modeltype, inputsize)
train_dataloader = DataLoader(train_dataset, batch_size=256,
                        shuffle=True, num_workers = num_threads)

test_dataset = DatasetFactory.get_test_dataset(modeltype, inputsize)
test_dataloader = DataLoader(test_dataset, batch_size=256,
                        shuffle=False, num_workers = num_threads)

In [ ]:
from dataloader import display_pose
plt.figure(figsize=(20,140))

with torch.no_grad():  
    net.load_state_dict(torch.load(modelpath))
    net = net.eval()

    for i_batch, sample_batched in enumerate(tqdm(test_dataloader)):

        images = sample_batched['image'].to(device)
        poses = sample_batched['pose'].to(device)
        
        t0 = time.clock()
        coords, heatmaps = net(images)
        t1 = time.clock()
        print('average forward seconds=%f s| fps=%d'%((t1-t0),int(1/((t1-t0)))))
        for ids in range(40,60):
            display_pose(images[ids][:3,:,:],coords[ids],ids)
#             display_pose(images[ids][:3,:,:],poses[ids],ids)
        plt.show()
        break

In [ ]:
plt.imshow(heatmaps[0, 8].data.cpu().numpy())
plt.show()

In [2]:
from torchsummary import summary
import torch
from networks import *
from networks.senet import se_resnet
import os
device = torch.device("cuda" if True else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# model = se_resnet.senet18_ed().to(device)
# model = resnet18_ed().to(device)
# model = ShuffleNetV2.shufflenetv2_ed(width_mult=1.0).to(device)
model = MobileNetV2.mobilenetv2_ed(width_mult=1.0).to(device)
# model = nn.Sequential(*(list(model.children())[:-1]))
summary(model, input_size=(3, 224, 224))
# print(list(model.children())[:-1])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             864
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 32, 224, 224]           1,024
       BatchNorm2d-5         [-1, 32, 224, 224]              64
             ReLU6-6         [-1, 32, 224, 224]               0
            Conv2d-7         [-1, 32, 224, 224]             288
       BatchNorm2d-8         [-1, 32, 224, 224]              64
             ReLU6-9         [-1, 32, 224, 224]               0
           Conv2d-10         [-1, 16, 224, 224]             512
      BatchNorm2d-11         [-1, 16, 224, 224]              32
 InvertedResidual-12         [-1, 16, 224, 224]               0
           Conv2d-13         [-1, 96, 224, 224]           1,536
      BatchNorm2d-14         [-1, 96, 2

In [ ]:
h, w =56, 56
x = np.arange(0, h)
y = np.arange(0, w)
x, y = np.meshgrid(x, y)
# declare guass
guass_heatmap = np.zeros([16, h, w])
xy_pose = np.random.uniform(0,1,size=(16,2))
sigma = 1.0/h

for idx,(x0,y0) in enumerate(xy_pose):
    guass_heatmap[idx] = np.exp(-((x*1.0/w-x0)**2+(y*1.0/h-y0)**2)/(2.0*sigma**2))
    
print(guass_heatmap.shape)
plt.imshow(guass_heatmap[1])



In [ ]:
print(np.random.uniform(0,1,size=(2,3)))

In [ ]:
print(np.max(guass_heatmap[0]))
print(np.min(guass_heatmap[0]))
print(guass_heatmap[0])
xy_pose

In [ ]:
print(xy_pose)